Authentication

High level overview:    
&emsp;&emsp;Use API key to generate a refresh token\
&emsp;&emsp;Use refresh token to generate an access token\
&emsp;&emsp;Use access token to make calls to Tradestation's servers

Refresh tokens last forever by default
Access tokens last for 20 minutes

Quickly mention: can revoke all refresh tokens using https://signin.tradestation.com/oauth/revoke endpoint

In [ ]:
import json
import requests
import os

# authentication requires your API credentials
CLIENT_ID = os.environ.get('CLIENT_ID') # api key
CLIENT_SECRET = os.environ.get('CLIENT_SECRET') # secret

In [ ]:
# generate a refresh token
# run this code block and then copy/paste the login URL into your brower and login with your TradeStation credentials
print(f'https://signin.tradestation.com/authorize?response_type=code&client_id={CLIENT_ID}&audience=https%3A%2F%2Fapi.tradestation.com&redirect_uri=http%3A%2F%2Flocalhost%3A3000&scope=openid%20MarketData%20profile%20ReadAccount%20Trade%20offline_access%20Matrix%20OptionSpreads')

In [ ]:
# when you login, you will get a "code" returned in the URL
# paste the "code" into this variable assignment statement and run this block
CODE = ''

# this request will get a new access token and refresh token
# if desired, you can paste the refresh token above and rerun that code block
# then after that, you can simply run the next code block anytime you need a new access token
url = "https://signin.tradestation.com/oauth/token"

payload=f'grant_type=authorization_code&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&code={CODE}&redirect_uri=http%3A%2F%2Flocalhost%3A3000'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)
response_data = response.json()
REFRESH_TOKEN = response_data['refresh_token']
print('refresh_token: ', REFRESH_TOKEN)

In [ ]:
REFRESH_TOKEN = os.environ.get('REFRESH_TOKEN') # your refresh token

In [ ]:
# this step will get a new access token using your refresh token when this function is called
def get_access_token():
    url = "https://signin.tradestation.com/oauth/token"

    payload=f'grant_type=refresh_token&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&refresh_token={REFRESH_TOKEN}'
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    response_data = response.json()
    return response_data['access_token']

Simulation vs live connection

To use sim, you use sim-api. To use live, you just use api.

For example, for retrieving account information, the urls would look like the following:\
&emsp;&emsp;SIM: https://sim-api.tradestation.com/v3/brokerage/accounts \
&emsp;&emsp;Live: https://api.tradestation.com/v3/brokerage/accounts

In [ ]:
core_url = "https://sim-api.tradestation.com"

In [ ]:
# get market data - snapshot
access_token = get_access_token()

url = f"{core_url}/v3/marketdata/barcharts/@ES"

headers = {"Authorization": f'Bearer {access_token}'}

params = {
    "unit": "Minute",
    "interval": "5",
    "barsback": "10"
}

response = requests.request("GET", url, headers=headers, params=params)

response.json()

In [ ]:
# get market data - streaming
access_token = get_access_token()

url = f"{core_url}/v3/marketdata/stream/barcharts/@ES"

headers = {"Authorization": f'Bearer {access_token}'}

params = {
    "unit": "Minute",
    "interval": "5",
    "barsback": "10"
}

response = requests.request("GET", url, headers=headers, params=params, stream=True)

#print(response.text)
for line in response.iter_lines():
    if line:
        print(line)

In [ ]:
# get symbol details
access_token = get_access_token()

url = f"{core_url}/v3/marketdata/symbols/@ES"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)

json_data = response.json()
print(json.dumps(json_data, indent=4, sort_keys=False))

In [ ]:
# get accounts

access_token = get_access_token() # get a new access token
url = f"{core_url}/v3/brokerage/accounts"

headers = {'Authorization': f'Bearer {access_token}' }

response = requests.request("GET", url, headers=headers)
json_data = response.json()
print(json.dumps(json_data, indent=4, sort_keys=False))

In [ ]:
account_id = os.environ.get('ACCOUNT_ID')

In [ ]:
# get balances real time
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/accounts/{account_id}/balances"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)

print(response.text)

In [ ]:
# get balance beginning of day
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/accounts/{account_id}/bodbalances"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)

print(response.text)

In [ ]:
# get historical orders
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/accounts/{account_id}/historicalorders"

querystring = {"since":"2024-09-01"}

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
# get current orders - today's orders + active orders
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/accounts/{account_id}/orders"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)
json_data = response.json()
print(json.dumps(json_data, indent=4, sort_keys=False))

In [ ]:
# stream orders
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/stream/accounts/{account_id}/orders"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [ ]:
# get positions
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/accounts/{account_id}/positions"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)

print(response.text)

In [ ]:
# stream positions
access_token = get_access_token()

url = f"{core_url}/v3/brokerage/stream/accounts/{account_id}/positions"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [ ]:
# confirm order
access_token = get_access_token()

url = f"{core_url}/v3/orderexecution/orderconfirm"

payload = {
    "AccountID": account_id,
    "Symbol": "ESZ24",
    "Quantity": "1",
    "OrderType": "Market",
    "TradeAction": "SELL",
    "TimeInForce": {"Duration": "DAY"},
    "Route": "Intelligent"
}
headers = {
    "content-type": "application/json",
    "Authorization": f'Bearer {access_token}'
}

response = requests.request("POST", url, json=payload, headers=headers)

#print(response.text)
json_data = response.json()
print(json.dumps(json_data, indent=4, sort_keys=False))

In [ ]:
# place order
access_token = get_access_token()

url = f"{core_url}/v3/orderexecution/orders"

payload = {
    "AccountID": account_id,
    "Symbol": "ESZ24",
    "Quantity": "1",
    "OrderType": "Market",
    "TradeAction": "SELL",
    "TimeInForce": {"Duration": "DAY"},
    "Route": "Intelligent"
}
headers = {
    "content-type": "application/json",
    "Authorization": f'Bearer {access_token}'
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

In [ ]:
# cancel order
access_token = get_access_token()

order_id = 
url = f"{core_url}/v3/orderexecution/orders/{order_id}"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("DELETE", url, headers=headers)

print(response.text)

In [ ]:
# get routes
access_token = get_access_token()

url = f"{core_url}/v3/orderexecution/routes"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)


json_data = response.json()
print(json.dumps(json_data, indent=4, sort_keys=False))

In [ ]:
# stream tick bars
access_token = get_access_token()

# {symbol}/{interval}/{barsBack}
url = f"{core_url}/v2/stream/tickbars/@ES/100/5"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [ ]:
# automated contract roll for futures
symbol = "ESU24" 
position = 1

access_token = get_access_token()

url = f"{core_url}/v3/marketdata/symbols/@ES"

headers = {"Authorization": f'Bearer {access_token}'}

response = requests.request("GET", url, headers=headers)

json_data = response.json()

top_month_contract = json_data['Symbols'][0]['Underlying']
print(top_month_contract)

# if the contract we're trading does not match the top month
if symbol != top_month_contract:
    # if we have a position
    if position != 0:
        # exit the position in the old contract
        placeOrder(symbol, -1 * position)
        # update our contract to the top month
        symbol = top_month_contract
        # re-open our position in the new contract
        placeOrder(symbol, position)
    else:
        # update our contract to the top month
        symbol = top_month_contract